In [1]:
%config Completer.use_jedi = False
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [16]:
#! /usr/bin/python3

import sys
import pennylane as qml
import numpy as np
import sklearn as skl
import autograd.numpy as np
import itertools
import time

def train_circuit(circuit,n_params,X_train,Y_train,rate_type='accuracy',**kwargs):
    """Develop and train your very own variational quantum classifier.

    Use the provided training data to train your classifier. The code you write
    for this challenge should be completely contained within this function
    between the # QHACK # comment markers. The number of qubits, choice of
    variational ansatz, cost function, and optimization method are all to be
    developed by you in this function.

    Args:
        circuit (qml.QNode): A circuit that you want to train
        X_train (np.ndarray): An array of floats of size (M, n) to be used as training data.
        Y_train (np.ndarray): An array of size (M,) which are the categorical labels
            associated to the training data. The categories are labeled by -1, 0, and 1.
        X_test (np.ndarray): An array of floats of (B, n) to serve as testing data.
        kwargs: hyperparameters for the training (steps, batch_size, learning_rate)

    Returns:
        (p,i,e,w): The number of parameters, the inference time (time to evaluate the accuracy), error rate (accuracy on the test set)
    """

    # Use this array to make a prediction for the labels of the data in X_test
    from autograd.numpy import exp,tanh

    def hinge_loss(labels, predictions,type='L2'):
        loss = 0
        for l, p in zip(labels, predictions):
            if type=='L1':
                loss = loss + np.abs(l - p) # L1 loss
            elif type=='L2':
                loss = loss + (l - p) ** 2 # L2 loss
        loss = loss/len(labels)
        return loss

    def accuracy(labels, predictions):

        loss = 0
        tol = 0.05
        #tol = 0.1
        for l, p in zip(labels, predictions):
            if abs(l - p) < tol:
                loss = loss + 1
        loss = loss / len(labels)

        return loss

    def cost_fcn(params,circuit=None,ang_array=[], actual=[]):
        '''
        use MAE to start
        '''
        labels = {2:-1,1:1,0:0}
        n = len(ang_array[0])
        w = params[-n:]
        theta = params[:-n]
        predictions = [2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, angles=x)))))- 1. for x in ang_array]
        return hinge_loss(actual, predictions)

    var = np.hstack((np.zeros(n_params),5*np.random.random(X_train.shape[1])-2.5))
    steps = kwargs['s']
    batch_size = kwargs['batch_size']
    num_train = len(Y_train)
    validation_size = 3*kwargs['batch_size']
    opt = qml.AdamOptimizer(kwargs['learning_rate'])
    start = time.time()
    for _ in range(steps):
        batch_index = np.random.randint(0, num_train, (batch_size,))
        X_train_batch = X_train[batch_index]
        Y_train_batch = Y_train[batch_index]
        var,cost = opt.step_and_cost(lambda v: cost_fcn(v, circuit,X_train_batch, Y_train_batch), var)
    end = time.time()
    cost_time = (end-start)

    w = var[-X_train.shape[1]:]
    theta = var[:-X_train.shape[1]]

    if rate_type =='accuracy':
        validation_batch = np.random.randint(0, num_train, (validation_size,))
        X_validation_batch = X_train[validation_batch]
        Y_validation_batch = Y_train[validation_batch]
        start = time.time() # add in timeit function from Wbranch
        predictions=[int(np.round(2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, angles=x)))))- 1.,0)) for x in X_validation_batch]
        end = time.time()
        inftime = (end-start)/len(X_validation_batch)
        err_rate = 1.0 - accuracy(predictions,Y_validation_batch)
    elif rate_type=='batch_cost':
        err_rate = cost
        inftime = cost_time
    # QHACK #
    W_ = np.abs((100.-len(var))/(100.))*np.abs((100.-inftime)/(100.))*(1./err_rate)
    return len(var),inftime,err_rate,W_,var


def classify_data(X_train,Y_train,X_test,Y_test,**kwargs):
    """Develop and train your very own variational quantum classifier.

    Use the provided training data to train your classifier. The code you write
    for this challenge should be completely contained within this function
    between the # QHACK # comment markers. The number of qubits, choice of
    variational ansatz, cost function, and optimization method are all to be
    developed by you in this function.

    Args:
        X_train (np.ndarray): An array of floats of size (250, 3) to be used as training data.
        Y_train (np.ndarray): An array of size (250,) which are the categorical labels
            associated to the training data. The categories are labeled by -1, 0, and 1.
        X_test (np.ndarray): An array of floats of (50, 3) to serve as testing data.

    Returns:
        str: The predicted categories of X_test, converted from a list of ints to a
            comma-separated string.
    """

    # Use this array to make a prediction for the labels of the data in X_test
    predictions = []

    # QHACK #

    from autograd.numpy import exp,tanh

    def statepreparation(a):
        qml.templates.embeddings.AngleEmbedding(a, wires=range(3), rotation='Y')

    def layer(W):
        qml.templates.layers.BasicEntanglerLayers(W, wires=range(3), rotation=qml.ops.RY)

    def hinge_loss(labels, predictions,type='L2'):
        loss = 0
        for l, p in zip(labels, predictions):
            if type=='L1':
                loss = loss + np.abs(l - p) # L1 loss
            elif type=='L2':
                loss = loss + (l - p) ** 2 # L2 loss
        loss = loss/len(labels)
        return loss

    def accuracy(labels, predictions):

        loss = 0
        tol = 0.05
        #tol = 0.1
        for l, p in zip(labels, predictions):
            if abs(l - p) < tol:
                loss = loss + 1
        loss = loss / len(labels)

        return loss

    def cost_fcn(params,circuit=None,ang_array=[], actual=[]):
        '''
        use MAE to start
        '''
        labels = {2:-1,1:1,0:0}
        w = params[-3:]
        theta = params[:-3]
        predictions = [2.*(1.0/(1.0+exp(np.dot(-w,circuit(theta, angles=x)))))- 1. for x in ang_array]
        return hinge_loss(actual, predictions)

    dev = qml.device("default.qubit", wires=3)
    @qml.qnode(dev)
    def inside_circuit(params,angles=None):
        statepreparation(angles)
        W= np.reshape(params,(len(params)//3,3))
        layer(W)
        return qml.expval(qml.PauliZ(0)),qml.expval(qml.PauliZ(1)),qml.expval(qml.PauliZ(2))


    var = np.hstack((np.zeros(6),5*np.random.random(3)-2.5))
    steps = kwargs['s']
    batch_size = kwargs['batch_size']
    num_train = len(Y_train)
    validation_size = int(num_train//2)
    opt = qml.AdamOptimizer(kwargs['learning_rate'])

    for _ in range(steps):
        batch_index = np.random.randint(0, num_train, (batch_size,))
        X_train_batch = X_train[batch_index]
        Y_train_batch = Y_train[batch_index]

        var,cost = opt.step_and_cost(lambda v: cost_fcn(v, inside_circuit,X_train_batch, Y_train_batch), var)

    # need timing values from computing predictions

    
    theta = var[:-3]
    w = var[-3:]
    start = time.time() # add in timeit function from Wbranch
    predictions=[int(np.round(2.*(1.0/(1.0+exp(np.dot(-w,inside_circuit(theta, angles=x)))))- 1.,0)) for x in X_test]
    end = time.time()
    inftime = end-start
    err_rate = 1.0 - accuracy(predictions,Y_test)
    # QHACK #
    W_ = len(var)*inftime*(1./err_rate)
    return len(var),inftime,err_rate,W_

## Import data from sklearn

In [3]:
from sklearn import datasets

In [4]:
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)

In [5]:
X_train = noisy_circles[0][:1450]
Y_train = noisy_circles[1][:1450]
X_test = noisy_circles[0][50:]
Y_test = noisy_circles[1][50:]

### Try running a loop over some hyper_parameters

This uses the circuit classifer built during the Challenge -- the circuit and QNode is constructed inside the function `classify_data`

The following characteristics are hard wired inside the function `classify_data`:
* number of qubits (3)
* number of rotation gates (6)
* the initialization used for the rotations and weights (rotations intialized with 0, weights initialized with random numbers drawn uniformly from $[-2.5,2.5]$
* the rotation gates that are trained (RY)
* the rotation gates used in the `AngleEmbeddding` (RY)
* the optimizer (`AdamOptimizer`)

The following parameters are passed as keywords:
* `s` (the number of steps to train for)
* `batch_size` (the batch size used in training)
* `learning_rate` (the initial learning rate for Adam)

As in (de Wynter 2020) we only train each circuit for a few steps (here I'm using 10).  In (de Wynter 2020) the error rate surrogate is defined using a loss function evaluated over a subset of the data -- here I'm using the accuracy of assigned labels over the test data.  The full number of samples that I generated for the dataset is given by `n_samples` (above).  I've split that data in to train,test sets. 

In [6]:
batch_sets = [2,4,8]
learning_rates = [0.01,0.05]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
for idx,sdx in hyperparameters:
    print(idx,sdx,classify_data(X_train,Y_train,X_test,Y_test,s=3,batch_size=idx,learning_rate=sdx))

[(2, 0.01), (2, 0.05), (4, 0.01), (4, 0.05), (8, 0.01), (8, 0.05)]
2 0.01 (9, 8.796605110168457, 0.49862068965517237, 158.77689721673354)
2 0.05 (9, 8.42807388305664, 0.49862068965517237, 152.1249850261261)
4 0.01 (9, 9.946794986724854, 0.5013793103448276, 178.5497587025575)
4 0.05 (9, 8.412147998809814, 0.7082758620689655, 106.89243562265636)
8 0.01 (9, 8.332909107208252, 0.49862068965517237, 150.4072805657921)
8 0.05 (9, 8.277477979660034, 0.5013793103448276, 148.5847147655618)


In [7]:
#uncommenting and running this cell should cause an error
#inside_circuit 

In [8]:
X_train.shape[1]

2

### Try running a loop over some hyper_parameters

This time use a circuit (QNode) that is created outside the function and passed as an argument

Things that are still hard-wired inside the `train_circuit` function:
* Optimizer choice
* Initialization choice (same as above)


In [9]:
QUBITS = X_train.shape[1]
def variational_circuit(params,angles=None):
    qml.templates.embeddings.AngleEmbedding(angles, wires=range(QUBITS), rotation='Y') # replace with more general embedding
    W= np.reshape(params,(len(params)//QUBITS,QUBITS))
    qml.templates.layers.BasicEntanglerLayers(W, wires=range(QUBITS), rotation=qml.ops.RY)
    return [qml.expval(qml.PauliZ(idx)) for idx in range(QUBITS)]
dev = qml.device("default.qubit",wires=QUBITS)
# Instantiate the QNode
outside_circuit = qml.QNode(func=variational_circuit,device=dev)


In [10]:
# redefine variational_circuit to take architecture parameters

In [11]:
outside_circuit #this should not cause an error

<QNode (differentiable): device='default.qubit', func=variational_circuit, wires=2, interface=autograd>

In [12]:
outside_circuit(np.zeros(10),angles=[0.5,-0.5]) #this should return something

array([0.87758256, 0.77015115])

In [24]:
def loop_over_hyperparameters(circuit,n_params,X_train,Y_train,batch_sets,learning_rates,**kwargs):
    """
    together with the function train_circuit(...) this executes lines 7-8 in the Algorithm 1 pseudo code of (de Wynter 2020)
    """
    hyperparameter_space = list(itertools.product(batch_sets,learning_rates))
    Wmax = 0.0
    s = kwargs.get('s', None)
    rate_type = kwargs.get('rate_type',None)
    
    for idx,sdx in hyperparameters:
        p,i,er,wtemp,weights=train_circuit(circuit,n_params,X_train,Y_train,X_test,Y_test,s=s,batch_size=idx,rate_type=rate_type,learning_rate=sdx)
        if wtemp>=Wmax:
            Wmax=wtemp
            saved_weights = weights
    return Wmax,saved_weights

In [19]:
batch_sets = [2,4,8,16,32]
learning_rates = [0.001,0.005,0.01,0.05]
hyperparameters = list(itertools.product(batch_sets,learning_rates))
print(hyperparameters)
results = {}
Wmax = 0.0
for idx,sdx in hyperparameters:
    p,i,er,wtemp,weights=train_circuit(outside_circuit,10,X_train,Y_train,X_test,Y_test,s=5,batch_size=idx,rate_type='accuracy',learning_rate=sdx)
    print(p,i,er,wtemp,weights)
    if wtemp>=Wmax:
        Wmax=wtemp
        saved_weights = weights
        output = (idx,sdx,p,i,er)
print("Max W coef: ", Wmax)
print("saved weights: ",saved_weights)
print("hyperparameters: ",output[0],output[1])
print("variables (p,i,er): ",p,i,er)

[(2, 0.001), (2, 0.005), (2, 0.01), (2, 0.05), (4, 0.001), (4, 0.005), (4, 0.01), (4, 0.05), (8, 0.001), (8, 0.005), (8, 0.01), (8, 0.05), (16, 0.001), (16, 0.005), (16, 0.01), (16, 0.05), (32, 0.001), (32, 0.005), (32, 0.01), (32, 0.05)]
12 0.006519476572672526 0.33333333333333337 2.639827885818481 [-2.77445101e-03  1.96276072e-03  2.94774669e-03  2.06786760e-03
 -2.77445691e-03  1.96196718e-03  2.95038428e-03  2.06785333e-03
 -2.77446220e-03  1.96117652e-03  1.06761315e+00 -2.31490758e+00]
12 0.007224122683207194 0.33333333333333337 2.6398092831611626 [-0.01452297 -0.02280676 -0.0180015  -0.02233194 -0.01452283 -0.02279085
 -0.01827237 -0.02233205 -0.01452281 -0.02277464  0.27661759 -1.96738303]
12 0.006486852963765462 0.6666666666666667 1.3199143735408783 [ 0.02628975 -0.02976622 -0.02810189 -0.02999918  0.02629207 -0.0298603
 -0.02803198 -0.02999462  0.02629511 -0.02995237  2.26438751  0.84421112]
12 0.006497343381245931 0.16666666666666663 5.279656940269472 [-0.05189296  0.0717931

In [23]:
loop_over_hyperparameters(outside_circuit,4,X_train,Y_train,batch_sets=[2,4,8,16,32],learning_rates=[0.001,0.005,0.01,0.05],s=5,rate_type='accuracy')

(3.00790045589606,
 array([0.01583075, 0.01626606, 0.01633701, 0.01636963, 0.16125688,
        1.59823   ]))

### Pull in more detailed circuit design